In [1]:
# Environment preparation -------------------------------------------------
import os
import numpy as np
from typing import Any, List, Dict, Tuple
import pandas as pd
import random
import matplotlib.pyplot as plt
import copy
from collections import Counter
from collections import defaultdict
from operator import itemgetter
from itertools import groupby
import operator
from pprint import pprint

In [2]:
# type-setting
Array = np.ndarray

In [3]:
path = "/Users/born/Documents/RIDER3/client/public/stimuli"

# we only want png file
stimuli_name_list = [el for el in os.listdir(path) if os.path.isfile(os.path.join(path, el)) if not(el.endswith("white.png")) and el.endswith('.png')]#one image is called white and cannot be used as a stimulus.
print(type(stimuli_name_list))
n_images = len(stimuli_name_list)
print("There are", n_images, "images in my image folder right now.")
stimuli_name_list

<class 'list'>
There are 112 images in my image folder right now.


['pylon01.png',
 'pipewrench.png',
 'redfox.png',
 'soapdispenser03b.png',
 'stackingring01b.png',
 'desktopcomputer.png',
 'chessknight03b.png',
 'boxingglove01.png',
 'honeybee.png',
 'toad.png',
 'jeep.png',
 'nyala.png',
 'wheelbarrow01.png',
 'toyhockeystick02a.png',
 'pintofbeer.png',
 'radio03a.png',
 'graduationcap.png',
 'razor04a.png',
 'flipflop01a.png',
 'sailboat.png',
 'rollerblade.png',
 'dragonfly01.png',
 'tabascosauce.png',
 'fryingpan01.png',
 'wateringcan.png',
 'saw02b.png',
 'motoroilbottle04.png',
 'hat04a.png',
 'flamingo.png',
 'bluejay.png',
 'witchhat.png',
 'lynx01.png',
 'mailbox02.png',
 'lock02a.png',
 'zebra.png',
 'armadillo.png',
 'lightbulb01.png',
 'handfan01b.png',
 'thumbtack02b.png',
 'duck01.png',
 'candelabra.png',
 'handbrush02.png',
 'gorilla.png',
 'clownfish.png',
 'paintroller02.png',
 'shoppingcart.png',
 'acousticguitar02.png',
 'snowglobe.png',
 'officechair02.png',
 'windsurfboard.png',
 'outdoorheater.png',
 'megaphone.png',
 'buoy.png

In [4]:
# I want the item names to be preceeded by "stimuli/", which makes it easier to use them as excel file later on
string = 'stimuli/'
stimuli_names = [string + x for x in stimuli_name_list]
stimuli_names[1]

'stimuli/pipewrench.png'

In [5]:
def sample_stimuli(stimuli_names: List[str], num_stimuli: int) -> List[str]:
    """Sample <num_stimuli> stimuli from the set of all stimuli without replacement."""
    sampled_stimuli = np.random.choice(a=stimuli_names, size=num_stimuli, replace=False).tolist()
    assert len(set(sampled_stimuli)) == len(sampled_stimuli), '\nStimuli must be unique.\n'
    return sampled_stimuli

In [6]:
def add_orientations_to_stimuli(sampled_stimuli: List[str], orientations: List[str]):
    """Add the orientation directly to the string of the stimuli to enable fixed connection between stimuli and orientations"""
    stimuli_and_orientations = list()
    for stim in sampled_stimuli:
        orientation = np.random.choice(a=possible_orientations, size=1, replace=False)
        sampled_stimuli = [stim + str(orientation)]
        stimuli_and_orientations.append(sampled_stimuli)
    return stimuli_and_orientations

In [7]:
def remove_stimuli(stimuli: List[str], sampled_stimuli: List[str]) -> None:
    """Remove stimuli that were sampled for a single trial from the set of possible exp. stimuli."""
    for stimulus in sampled_stimuli:
        stimuli.pop(stimuli.index(stimulus))

In [8]:
def partition_stimuli_into_experimental_conditions(
    stimuli: List[str],
    num_conditions: int,
    num_trials_per_condition: int
) -> Dict[str, List[str]]:
    """Partition stimuli into the five different conditions."""
    condition_partitioning = {}
    stimuli_copy = copy.deepcopy(stimuli)
    for i in range(1, num_conditions+1):
        # one stimulus is needed per condition
        num_stimuli = num_trials_per_condition
        condition_stimuli = np.random.choice(a=stimuli_copy, size=num_stimuli, replace=False).tolist()
        remove_stimuli(stimuli=stimuli_copy, sampled_stimuli=condition_stimuli)
        condition_partitioning[f'condition{i}'] = condition_stimuli
    print(len(stimuli))
    return condition_partitioning

In [9]:
def rotate_stimuli(
    condition_partitioning: Dict[str, List[str]],
    num_rotation_stimuli: int
) -> Dict[str, List[str]]:
    """Rotate stimuli through conditions by <num_rotation_stimuli>."""
    condition1_stimuli = condition_partitioning['condition1']
    condition2_stimuli = condition_partitioning['condition2']
    condition3_stimuli = condition_partitioning['condition3']
    condition4_stimuli = condition_partitioning['condition4']
    condition5_stimuli = condition_partitioning['condition5']
    condition6_stimuli = condition_partitioning['condition6']
    condition7_stimuli = condition_partitioning['condition7']
    condition8_stimuli = condition_partitioning['condition8']
    
    condition_partitioning['condition1'] = condition8_stimuli[:num_rotation_stimuli]
    condition_partitioning['condition2'] = condition1_stimuli[:num_rotation_stimuli]
    condition_partitioning['condition3'] = condition2_stimuli[:num_rotation_stimuli]
    condition_partitioning['condition4'] = condition3_stimuli[:num_rotation_stimuli]
    condition_partitioning['condition5'] = condition4_stimuli[:num_rotation_stimuli]
    condition_partitioning['condition6'] = condition5_stimuli[:num_rotation_stimuli]
    condition_partitioning['condition7'] = condition6_stimuli[:num_rotation_stimuli]
    condition_partitioning['condition8'] = condition7_stimuli[:num_rotation_stimuli]
   
    return condition_partitioning

In [10]:
def sample_condition(stimuli: List[str], experiment_condition: str) -> Dict[str, str]:
    """Sample a single condition."""
    # initialize an empty dictionary to store all variables necessary for a single trial
    condition = {}
    if(experiment_condition == 'condition1'):
        # draw a single stimulus for presenting images and testing retrieval 
        sampled_stimuli = random.choices(stimuli, k=1)
        condition['image1'] = sampled_stimuli[0]
        condition['cue'] = '0'
        condition['test1']  = sampled_stimuli[0]
        condition['trial_type'] = 'attended_1'
    elif(experiment_condition == 'condition2'):
        # draw a single stimulus for presenting images and testing retrieval 
        sampled_stimuli = random.choices(stimuli, k=1)
        condition['image2'] = sampled_stimuli[0]
        condition['trial_type'] = 'attended_1'
    elif(experiment_condition == 'condition3'):
        # draw a single stimulus for presenting images and testing retrieval 
        sampled_stimuli = random.choices(stimuli, k=1)
        condition['image1'] = sampled_stimuli[0]
        #condition['image2'] = sampled_stimuli[1]
        condition['cue'] = '1'
        condition['trial_type'] = 'attended_2'
    elif(experiment_condition == 'condition4'):
        # draw a single stimulus for presenting images and testing retrieval 
        sampled_stimuli = random.choices(stimuli, k=1)
        condition['image2'] = sampled_stimuli[0]
        condition['test1']  = sampled_stimuli[0]
        condition['trial_type'] = 'attended_2'
    elif(experiment_condition == 'condition5'):
        # draw a single stimulus for presenting images and testing retrieval 
        sampled_stimuli = random.choices(stimuli, k=1)
        condition['image1'] = sampled_stimuli[0]
        condition['cue'] = '1'
        condition['trial_type'] = 'attended_3'
    elif(experiment_condition == 'condition6'):
        # draw a single stimulus for presenting images and testing retrieval 
        sampled_stimuli = random.choices(stimuli, k=1)
        condition['image2'] = sampled_stimuli[0]
        condition['test1']  = sampled_stimuli[0]
        condition['trial_type'] = 'attended_3'
    elif(experiment_condition == 'condition7'):
        # draw a single stimulus for presenting images and testing retrieval 
        sampled_stimuli = random.choices(stimuli, k=1)
        condition['image1'] = sampled_stimuli[0]
        #condition['image2'] = sampled_stimuli[1]
        condition['cue'] = '1'
        condition['test1']  = sampled_stimuli[0]
        condition['trial_type'] = 'unattended'
    elif(experiment_condition == 'condition8'):
        # draw a single stimulus for presenting images and testing retrieval 
        sampled_stimuli = random.choices(stimuli, k=1)
        condition['image2'] = sampled_stimuli[0]
        condition['trial_type'] = 'unattended'
        # note that here is nothing following for condition5 because these stimuli are not shown in the WM experiment
        
    else:
        raise ValueError('\nThere are 5 conditions.\nUse condition1,condition2,condition3,condition4 or condition5 .\n')
    # remove sampled stimuli from the participant's set of stimuli so that they are not drawn again
    remove_stimuli(stimuli=stimuli, sampled_stimuli=sampled_stimuli)
    return condition

In [11]:
def sample_conditions(
    condition_order: List[str], 
    condition_sets: Dict[str, List[str]],
) -> List[str]:
    """Sample stimuli for all trials in a practice or experimental session."""
    sampled_conditions = [sample_condition(condition_sets[condition], condition) for condition in condition_order]
    return sampled_conditions

In [12]:
def participant_randomization(
    sessions: Dict[str, int],
    condition_sets: Dict[str, List[str]],
    conditions: List[str],
    num_conditions: int,
) -> List[str]:
    """Sample practice and experimental conditions for a single participant."""
    participant_conditions = []
    for session, number_conditions in sessions.items():
        # number of conditions per type
        num_conditions_per_type = number_conditions // len(conditions)
        # extend the list of condition types by <num_conditions_per_type>
        condition_order = conditions * num_conditions_per_type
        # randomly permute the order of condition types for a single session
        random.shuffle(condition_order)
        sampled_conditions = sample_conditions(condition_order,condition_sets)
        participant_conditions.extend(sampled_conditions)        
    #participant_conditions = pd.DataFrame(participant_conditions)
    print(num_conditions_per_type)
    print(len(conditions))
    print(number_conditions)
    print(condition_order)
    return participant_conditions

In [13]:
def stimuli_balancing(
    num_participants: int,
    sessions: Dict[str, int],
    condition_partitioning: Dict[str, List[str]],
    conditions: List[str],
) -> List[str]:
    """
    This is the main loop for balancing the experimental conditions. 
    Here, we sample both practice and experimental conditions for all participants.
    """
    num_conditions = len(conditions)
    particpant_randomizations = []
    for i in range(1, num_participants+1):
        if i > 1:
            # rotate stimuli through the three type sets by <num_rotation_stimuli> (currently 22)
            condition_partitioning = rotate_stimuli(condition_partitioning, num_rotation_stimuli = 14)
        # create copies of the three trial type sets so that we can remove stimuli 
        # from the three trial type sets without changing the original sets
        condition_partitioning_copy = copy.deepcopy(condition_partitioning)
        # sample practice and experimental trials for a single participant
        participant_conditions = participant_randomization(
            sessions=sessions,
            condition_sets=condition_partitioning_copy,
            conditions=conditions,
            num_conditions=num_conditions,
        )
        # add start orientations to conditions file
        #participant_trials['startOri1'] = start_orientations[:, 0]
        #participant_trials['startOri2'] = start_orientations[:, 1]
        #participant_trials['testOri1']  = start_orientations[:, 2]
        #participant_trials['testOri2']  = start_orientations[:, 3]
        
        # permute the order of the presented stimuli (i.e., shuffle rows) to counteract potential orientation biases
        #participant_conditions = participant_conditions.sample(frac=1, replace=False).reset_index(drop=True)
        particpant_randomizations.append(participant_conditions)
    return particpant_randomizations

In [14]:
# seed random number generator
rnd_seed = 40 #42
random.seed(rnd_seed)
np.random.seed(rnd_seed)

In [15]:
#### define experimental variables ####
num_stimuli = 112
num_practice_stimuli = 8
num_exp_stimuli = num_stimuli - num_practice_stimuli
num_participants = 8

ttypes = ['attended_1', 'attended_2', 'attended_3','unattended']
conditions = ['condition1','condition2','condition3','condition4', 'condition5','condition6','condition7','condition8']
num_practice_conditions = 0
num_exp_conditions = 112
num_total_conditions = num_practice_conditions + num_exp_conditions
sessions = {'practice': num_practice_conditions, 'experimental': num_exp_conditions}

stimulus_order = ['stim1','stim2','stim3','stim4','stim5']

# compute number of trials per trial type
num_condition_per_type =  14# num_total_trials // len(ttypes)

# define variables for orientations of the stimuli
min_orientation = 11.25
max_orientation = 371.25
step_size =  22.5

possible_orientations = np.arange(min_orientation, max_orientation, step_size)
possible_orientations = possible_orientations.tolist()
possible_orientations = possible_orientations * 7
len(possible_orientations)

112

In [6547]:
# sample stimuli without replacement
sampled_stimuli = sample_stimuli(stimuli_names=stimuli_names, num_stimuli=num_stimuli)
sampled_stimuli

['stimuli/zebra.png',
 'stimuli/redfox.png',
 'stimuli/pumpkin.png',
 'stimuli/scrubbingbrush05a.png',
 'stimuli/stapler03b.png',
 'stimuli/pipewrench.png',
 'stimuli/acorn.png',
 'stimuli/corkscrew03a.png',
 'stimuli/accordion02.png',
 'stimuli/chameleon.png',
 'stimuli/wateringcan.png',
 'stimuli/outdoorheater.png',
 'stimuli/photocopier.png',
 'stimuli/toad.png',
 'stimuli/peacock.png',
 'stimuli/monument.png',
 'stimuli/dromedary.png',
 'stimuli/rollerblade.png',
 'stimuli/chessknight03b.png',
 'stimuli/lightbulb01.png',
 'stimuli/bulldozer.png',
 'stimuli/microscope.png',
 'stimuli/needlenosepliers03b.png',
 'stimuli/dragonfly01.png',
 'stimuli/flipflop01a.png',
 'stimuli/blender.png',
 'stimuli/armadillo.png',
 'stimuli/pear01.png',
 'stimuli/graduationcap.png',
 'stimuli/wheelbarrow01.png',
 'stimuli/globe.png',
 'stimuli/comb02b.png',
 'stimuli/quartz.png',
 'stimuli/parrot01.png',
 'stimuli/mailbox02.png',
 'stimuli/handbrush02.png',
 'stimuli/stackingring01b.png',
 'stimuli/m

In [6548]:
# add orientations to stimuli that can be extracted to independant variables later
sampled_stimuli = add_orientations_to_stimuli(sampled_stimuli = sampled_stimuli, orientations = possible_orientations)
# flatten List of Lists Using a List Comprehension
sampled_stimuli = [item for sublist in sampled_stimuli for item in sublist]

In [6549]:
# partition the stimuli into three disjoint sets, one for each trial type
condition_partitioning = partition_stimuli_into_experimental_conditions(
    stimuli=sampled_stimuli,
# we need to partition the stimuli across all 5 conditions, although trial type 3 test 2 will be "invisible"
    num_conditions= 8,
    num_trials_per_condition=14
)
condition_partitioning

112


{'condition1': ['stimuli/boxingglove01.png[236.25]',
  'stimuli/makeupbrush04.png[191.25]',
  'stimuli/backpack01a.png[11.25]',
  'stimuli/chessknight03b.png[123.75]',
  'stimuli/saw02b.png[168.75]',
  'stimuli/dragonfly01.png[11.25]',
  'stimuli/pear01.png[123.75]',
  'stimuli/zebra.png[123.75]',
  'stimuli/nail.png[236.25]',
  'stimuli/globe.png[101.25]',
  'stimuli/officechair02.png[303.75]',
  'stimuli/paintroller02.png[213.75]',
  'stimuli/hen.png[146.25]',
  'stimuli/pumpkin.png[33.75]'],
 'condition2': ['stimuli/penguin.png[326.25]',
  'stimuli/rollerblade.png[168.75]',
  'stimuli/fireextinguisher02.png[11.25]',
  'stimuli/gorilla.png[168.75]',
  'stimuli/cupcake.png[123.75]',
  'stimuli/comb02b.png[258.75]',
  'stimuli/scrubbingbrush05a.png[146.25]',
  'stimuli/pylon01.png[191.25]',
  'stimuli/stackingring01b.png[281.25]',
  'stimuli/microscope.png[236.25]',
  'stimuli/fryingpan01.png[303.75]',
  'stimuli/badmintonracket.png[281.25]',
  'stimuli/duck01.png[281.25]',
  'stimuli/

In [6550]:
# get all participant condition randomizations
participant_randomizations = stimuli_balancing(
    num_participants=8,
    sessions=sessions, 
    condition_partitioning= condition_partitioning,
    conditions=conditions, 
)


14
8
112
['condition5', 'condition1', 'condition5', 'condition6', 'condition7', 'condition3', 'condition4', 'condition8', 'condition7', 'condition7', 'condition5', 'condition6', 'condition6', 'condition7', 'condition6', 'condition5', 'condition2', 'condition8', 'condition8', 'condition7', 'condition2', 'condition7', 'condition8', 'condition3', 'condition8', 'condition4', 'condition1', 'condition7', 'condition1', 'condition3', 'condition8', 'condition6', 'condition6', 'condition5', 'condition2', 'condition4', 'condition6', 'condition5', 'condition8', 'condition4', 'condition3', 'condition3', 'condition7', 'condition8', 'condition6', 'condition3', 'condition1', 'condition1', 'condition4', 'condition4', 'condition8', 'condition6', 'condition4', 'condition8', 'condition2', 'condition1', 'condition2', 'condition1', 'condition4', 'condition1', 'condition3', 'condition3', 'condition7', 'condition2', 'condition5', 'condition5', 'condition2', 'condition7', 'condition4', 'condition4', 'condition

In [6551]:
def sort_conditions_in_trial_types(
    num_participants: int,
    particpant_randomizations: List[str],
)-> List[str]:
    """Sort conditions in trial type 1 (baseline) and 3"""
    trial_types = itemgetter('trial_type')
    participant_grouped_conditions = {}
    # outer loop: loop over each randomization
    for i, randomization in enumerate(particpant_randomizations, start=1):
        trial_type_grouping = defaultdict(list)
         # inner loop: loop over every condition in current subject's randomization
        for condition in randomization:
            trial_type_grouping[trial_types(condition)].append(condition)
        participant_grouped_conditions[f'participant_{i:02d}'] = trial_type_grouping
        
    """
    trial_types = itemgetter('trial_type')
    trial_type_grouping = defaultdict(lambda: defaultdict(list))
    for i, randomization in enumerate(particpant_randomizations, start=1):
        for condition in randomization:
            trial_type_grouping[f'participant_{i:02d}'][trial_types(condition)].append(condition)
    """
    return participant_grouped_conditions

In [6552]:
sorted_conditions = sort_conditions_in_trial_types(num_participants = 8 ,particpant_randomizations=participant_randomizations)
sorted_conditions

{'participant_01': defaultdict(list,
             {'attended_3': [{'image1': 'stimuli/parrot01.png[258.75]',
                'cue': '1',
                'trial_type': 'attended_3'},
               {'image1': 'stimuli/quartz.png[348.75]',
                'cue': '1',
                'trial_type': 'attended_3'},
               {'image2': 'stimuli/monument.png[56.25]',
                'test1': 'stimuli/monument.png[56.25]',
                'trial_type': 'attended_3'},
               {'image1': 'stimuli/honeybee.png[56.25]',
                'cue': '1',
                'trial_type': 'attended_3'},
               {'image2': 'stimuli/motorcycle.png[326.25]',
                'test1': 'stimuli/motorcycle.png[326.25]',
                'trial_type': 'attended_3'},
               {'image2': 'stimuli/armadillo.png[326.25]',
                'test1': 'stimuli/armadillo.png[326.25]',
                'trial_type': 'attended_3'},
               {'image2': 'stimuli/dromedary.png[326.25]',
                

In [7427]:
# From here it goes participant per participant
attended_1       = sorted_conditions['participant_05']['attended_1']
attended_2       = sorted_conditions['participant_05']['attended_2']
attended_3       = sorted_conditions['participant_05']['attended_3']
unattended       = sorted_conditions['participant_05']['unattended']


# to construct the final trials I need to extract the trial variables that I need from the conditions
attended_1_stimuli_1      = [stimulus['image1'] for stimulus in attended_1 if 'image1' in stimulus]
attended_1_stimuli_2      = [stimulus['image2'] for stimulus in attended_1 if 'image2' in stimulus]
attended_1_stimuli_cue    = [stimulus['cue'] for stimulus in attended_1 if 'cue' in stimulus]
attended_1_stimuli_test    = [stimulus['test1'] for stimulus in attended_1 if 'test1' in stimulus]



def define_attended_1_trials(stimuli_image1: List[str],stimuli_image2: List[str],stimuli_cue: List[str]) -> List[str]:
    """Sample a single t2 trial"""
    # initialize an empty dictionary to store all variables necessary for a single trial
    attended_1 = {}
    for stimulus in range(len(stimuli_image1)):
        # draw a single stimulus for presenting images and testing retrieval
        sampled_stim_t1 = random.choices(stimuli_image1, k=1)
        sampled_stim_t2 = random.choices(stimuli_image2, k=1)
        cue = random.choices(stimuli_cue, k=1)
        attended_1['image1'] = sampled_stim_t1[0]
        attended_1['image2'] = sampled_stim_t2[0]
        attended_1['cue'] = cue[0]
        attended_1['test1'] = sampled_stim_t1[0]
        attended_1['trial_type'] = 'attended_1'
        attended_1.update(attended_1)
    remove_stimuli(stimuli=stimuli_image1, sampled_stimuli=sampled_stim_t1)
    remove_stimuli(stimuli=stimuli_image2, sampled_stimuli=sampled_stim_t2)
    remove_stimuli(stimuli=stimuli_cue, sampled_stimuli=cue)
    return attended_1

def sample_attended_1_trials(
    stimuli_image1: List[str],
    stimuli_image2: List[str],
    stimuli_cue: List[str],
    stimulus_order: List[str],
) -> List[str]:
    """Sample stimuli for all t2 trials in a practice or experimental session."""
    sampled_attended_1 = [define_attended_1_trials(stimuli_image1 = attended_1_stimuli_1, stimuli_image2 = attended_1_stimuli_2,stimuli_cue = attended_1_stimuli_cue) for stim in stimulus_order]
    return sampled_attended_1


stimulus_order = ['stim1','stim2','stim3','stim4','stim5','stim6','stim7','stim8','stim9','stim10','stim11','stim12','stim13','stim14']

attended_1_trials = sample_attended_1_trials(stimuli_image1=attended_1_stimuli_1,stimuli_image2=attended_1_stimuli_2,stimuli_cue=attended_1_stimuli_cue, stimulus_order =stimulus_order)





attended_2_stimuli_1 = []
attended_2_stimuli_2 = []
attended_2_stimuli_cue = []
attended_2_stimuli_test = []

# Iterate over each dictionary in the attended_2 list
for stimulus in attended_2:
    if 'image1' in stimulus:
        attended_2_stimuli_1.append(stimulus['image1'])
    if 'image2' in stimulus:
        attended_2_stimuli_2.append(stimulus['image2'])
    if 'cue' in stimulus:
        attended_2_stimuli_cue.append(stimulus['cue'])
    if 'test1' in stimulus:
        attended_2_stimuli_test.append(stimulus['test1'])

def define_attended_2_trials(stimuli_image1: List[str],stimuli_image2: List[str],stimuli_cue: List[str]) -> List[str]:
    """Sample a single t2 trial"""
    # initialize an empty dictionary to store all variables necessary for a single trial
    attended_2 = {}
    for stimulus in range(len(stimuli_image1)):
        # draw a single stimulus for presenting images and testing retrieval
        sampled_stim_t1 = random.choices(stimuli_image1, k=1)
        sampled_stim_t2 = random.choices(stimuli_image2, k=1)
        cue = random.choices(stimuli_cue, k=1)
        attended_2['image1'] = sampled_stim_t1[0]
        attended_2['image2'] = sampled_stim_t2[0]
        attended_2['cue'] = cue[0]
        attended_2['test1'] = sampled_stim_t2[0]
        attended_2['trial_type'] = 'attended_2'
        attended_2.update(attended_2)
    remove_stimuli(stimuli=stimuli_image1, sampled_stimuli=sampled_stim_t1)
    remove_stimuli(stimuli=stimuli_image2, sampled_stimuli=sampled_stim_t2)
    remove_stimuli(stimuli=stimuli_cue, sampled_stimuli=cue)
    return attended_2

def sample_attended_2_trials(
    stimuli_image1: List[str],
    stimuli_image2: List[str],
    stimuli_cue: List[str],
    stimulus_order: List[str],
) -> List[str]:
    """Sample stimuli for all t2 trials in a practice or experimental session."""
    sampled_attended_2 = [define_attended_2_trials(stimuli_image1 = attended_2_stimuli_1, stimuli_image2 = attended_2_stimuli_2,stimuli_cue = attended_2_stimuli_cue) for stim in stimulus_order]
    return sampled_attended_2


stimulus_order = ['stim1','stim2','stim3','stim4','stim5','stim6','stim7','stim8','stim9','stim10','stim11','stim12','stim13','stim14']

attended_2_trials = sample_attended_2_trials(stimuli_image1=attended_2_stimuli_1,stimuli_image2=attended_2_stimuli_2,stimuli_cue=attended_2_stimuli_cue, stimulus_order =stimulus_order)






attended_3_stimuli_1 = []
attended_3_stimuli_2 = []
attended_3_stimuli_cue = []
attended_3_stimuli_test = []

# Iterate over each dictionary in the attended_2 list
for stimulus in attended_3:
    if 'image1' in stimulus:
        attended_3_stimuli_1.append(stimulus['image1'])
    if 'image2' in stimulus:
        attended_3_stimuli_2.append(stimulus['image2'])
    if 'cue' in stimulus:
        attended_3_stimuli_cue.append(stimulus['cue'])
    if 'test1' in stimulus:
        attended_3_stimuli_test.append(stimulus['test1'])

def define_attended_3_trials(stimuli_image1: List[str],stimuli_image2: List[str],stimuli_cue: List[str]) -> List[str]:
    """Sample a single t2 trial"""
    # initialize an empty dictionary to store all variables necessary for a single trial
    attended_3 = {}
    for stimulus in range(len(stimuli_image1)):
        # draw a single stimulus for presenting images and testing retrieval
        sampled_stim_t1 = random.choices(stimuli_image1, k=1)
        sampled_stim_t2 = random.choices(stimuli_image2, k=1)
        cue = random.choices(stimuli_cue, k=1)
        attended_3['image1'] = sampled_stim_t1[0]
        attended_3['image2'] = sampled_stim_t2[0]
        attended_3['cue'] = cue[0]
        attended_3['test1'] = sampled_stim_t2[0]
        attended_3['trial_type'] = 'attended_3'
        attended_3.update(attended_3)
    remove_stimuli(stimuli=stimuli_image1, sampled_stimuli=sampled_stim_t1)
    remove_stimuli(stimuli=stimuli_image2, sampled_stimuli=sampled_stim_t2)
    remove_stimuli(stimuli=stimuli_cue, sampled_stimuli=cue)
    return attended_3

def sample_attended_3_trials(
    stimuli_image1: List[str],
    stimuli_image2: List[str],
    stimuli_cue: List[str],
    stimulus_order: List[str],
) -> List[str]:
    """Sample stimuli for all t2 trials in a practice or experimental session."""
    sampled_attended_3 = [define_attended_3_trials(stimuli_image1 = attended_3_stimuli_1, stimuli_image2 = attended_3_stimuli_2,stimuli_cue = attended_3_stimuli_cue) for stim in stimulus_order]
    return sampled_attended_3


stimulus_order = ['stim1','stim2','stim3','stim4','stim5','stim6','stim7','stim8','stim9','stim10','stim11','stim12','stim13','stim14']

attended_3_trials = sample_attended_3_trials(stimuli_image1=attended_3_stimuli_1,stimuli_image2=attended_3_stimuli_2,stimuli_cue=attended_3_stimuli_cue, stimulus_order =stimulus_order)





# Initialize empty lists
unattended_stimuli_1 = []
unattended_stimuli_2 = []
unattended_stimuli_cue = []
unattended_stimuli_test = []

# Iterate over each dictionary in the unattended list
for stimulus in unattended:
    if 'image1' in stimulus:
        unattended_stimuli_1.append(stimulus['image1'])
    if 'image2' in stimulus:
        unattended_stimuli_2.append(stimulus['image2'])
    if 'cue' in stimulus:
        unattended_stimuli_cue.append(stimulus['cue'])
    if 'test1' in stimulus:
        unattended_stimuli_test.append(stimulus['test1'])

def define_unattended_trials(stimuli_image1: List[str],stimuli_image2: List[str],stimuli_cue: List[str]) -> List[str]:
    """Sample a single t2 trial"""
    # initialize an empty dictionary to store all variables necessary for a single trial
    unattended = {}
    for stimulus in range(len(stimuli_image1)):
        # draw a single stimulus for presenting images and testing retrieval
        sampled_stim_t1 = random.choices(stimuli_image1, k=1)
        sampled_stim_t2 = random.choices(stimuli_image2, k=1)
        cue = random.choices(stimuli_cue, k=1)
        unattended['image1'] = sampled_stim_t1[0]
        unattended['image2'] = sampled_stim_t2[0]
        unattended['cue'] = cue[0]
        unattended['test1'] = sampled_stim_t1[0]
        unattended['trial_type'] = 'unattended'
        unattended.update(unattended)
    remove_stimuli(stimuli=stimuli_image1, sampled_stimuli=sampled_stim_t1)
    remove_stimuli(stimuli=stimuli_image2, sampled_stimuli=sampled_stim_t2)
    remove_stimuli(stimuli=stimuli_cue, sampled_stimuli=cue)
    return unattended

def sample_unattended_trials(
    stimuli_image1: List[str],
    stimuli_image2: List[str],
    stimuli_cue: List[str],
    stimulus_order: List[str],
) -> List[str]:
    """Sample stimuli for all t2 trials in a practice or experimental session."""
    sampled_unattended = [define_unattended_trials(stimuli_image1 = unattended_stimuli_1, stimuli_image2 = unattended_stimuli_2,stimuli_cue = unattended_stimuli_cue) for stim in stimulus_order]
    return sampled_unattended


stimulus_order = ['stim1','stim2','stim3','stim4','stim5','stim6','stim7','stim8','stim9','stim10','stim11','stim12','stim13','stim14']

unattended_stimuli = sample_unattended_trials(stimuli_image1=unattended_stimuli_1,stimuli_image2=unattended_stimuli_2,stimuli_cue=unattended_stimuli_cue, stimulus_order =stimulus_order)

trials = attended_3_trials + attended_2_trials+ attended_1_trials + unattended_stimuli
random.shuffle(trials)
len(stimulus_order)

# convert the list of trials into a pandas dataframe
participant_trials = pd.DataFrame(trials)

# Replace 0 with 1 and 1 with 2 in the 'cue' column
participant_trials['cue'] = participant_trials['cue'].replace({"0": "1", "1": "2"})

# Get the indices of rows where 'trial_type' is 'attended_3'
attended_3_indices = participant_trials.loc[participant_trials['trial_type'] == 'attended_3'].index

# Randomly choose 'image1' or 'image2' value for 'test1' in 'attended_3' rows
for index in attended_3_indices:
    random_column = np.random.choice(['image1', 'image2'])
    participant_trials.at[index, 'test1'] = participant_trials.at[index, random_column]
    
    
# Get the indices of rows where 'trial_type' is 'unattended'
unattended_indices = participant_trials.loc[participant_trials['trial_type'] == 'unattended'].index

# Randomly choose 'image1' or 'image2' value for 'test1' in 'attended_3' rows
for index in unattended_indices:
    random_column = np.random.choice(['image1', 'image2'])
    participant_trials.at[index, 'test1'] = participant_trials.at[index, random_column]


# Check for each row in all trial types
for index, row in participant_trials.iterrows():
    if row['test1'] == row['image1']:
        row['cue'] = 1
    elif row['test1'] == row['image2']:
        row['cue'] = 2

# Check for each row in the "unattended" trial type
unattended_indices = participant_trials.loc[participant_trials['trial_type'] == 'unattended'].index
for index in unattended_indices:
    if participant_trials.at[index, 'test1'] == participant_trials.at[index, 'image1']:
        participant_trials.at[index, 'cue'] = 2
    elif participant_trials.at[index, 'test1'] == participant_trials.at[index, 'image2']:
        participant_trials.at[index, 'cue'] = 1
        
        
# sample 6 practice trials 
size = 2        # sample size
replace = False  # with replacement
fn = lambda obj: obj.loc[np.random.choice(obj.index, size, replace),:]
training = participant_trials.groupby('trial_type', as_index=False).apply(fn)
training = training.sample(frac = 1)
#create final WM conditions file with 6 trainings trials in the beginning (two of each trial type)
trials_training = participant_trials['image1'].isin(training['image1'])
participant_trials.drop(participant_trials[trials_training].index, inplace = True)
trials_exp = participant_trials
print(len(trials_exp))
print(len(participant_trials))
participant_trials = pd.concat([training,participant_trials.loc[:]]).reset_index(drop=True)
print(len(participant_trials))

# generate ltm file
trials_exp = pd.DataFrame(participant_trials)
conditions_ltm_image1 = trials_exp[['trial_type','image1',]]
conditions_ltm_image2 = trials_exp[['trial_type','image2',]]
conditions_ltm_image2 = conditions_ltm_image2[conditions_ltm_image2['image2'] != '_']
conditions_ltm_image2 = conditions_ltm_image2.rename(columns={'image2': 'image1'})
conditions_ltm_ = pd.concat([conditions_ltm_image1,conditions_ltm_image2.loc[:]]).reset_index(drop=True)
conditions_ltm_['startOri_ltm'] = conditions_ltm_['image1'].str.split('[',1).str[1].str.strip()
conditions_ltm_['startOri_ltm'] = conditions_ltm_['startOri_ltm'].str.split(']',0).str[0].str.strip()
conditions_ltm_['ltm_image'] = conditions_ltm_['image1'].str.split('[',0).str[0].str.strip()
conditions_ltm_['startOri_ltm'] = conditions_ltm_['startOri_ltm'].astype(float)


# finish creating WM conditions file
# split orientation and object identity also in working memory experiment
participant_trials['startOri1'] = participant_trials['image1'].str.split('[',1).str[1].str.strip()
participant_trials['startOri1'] = participant_trials['startOri1'].str.split(']',0).str[0].str.strip()
participant_trials['startOri2'] = participant_trials['image2'].str.split('[',1).str[1].str.strip()
participant_trials['startOri2'] = participant_trials['startOri2'].str.split(']',0).str[0].str.strip()
participant_trials['image1'] = participant_trials['image1'].str.split('[',0).str[0].str.strip()
participant_trials['image2'] = participant_trials['image2'].str.split('[',0).str[0].str.strip()
participant_trials['test1'] = participant_trials['test1'].str.split('[',0).str[0].str.strip()


# converting string (numbers) to datatype float
participant_trials['startOri1'] = participant_trials['startOri1'].astype(float)
participant_trials['startOri2'] = participant_trials['startOri2'].astype(float)


# check that the presentation orientation of the two stimuli is not the same
participant_trials['orientation_match'] = np.where(participant_trials['startOri1'] == participant_trials['startOri2'], 'True', 'False')
#new_participant_trials['orientation_match'] = np.where(new_participant_trials['startOri1'] == new_participant_trials['startOri2'], 'True', 'False')
found = participant_trials[participant_trials['orientation_match'].str.contains('True')]
#found = new_participant_trials[new_participant_trials['orientation_match'].str.contains('True')]
print(found.count())



48
48
56
image1               0
image2               0
cue                  0
test1                0
trial_type           0
startOri1            0
startOri2            0
orientation_match    0
dtype: int64


In [7428]:
participant_trials

,image1,image2,cue,test1,trial_type,startOri1,startOri2,orientation_match
0,stimuli/quartz.png,stimuli/motorcycle.png,1,stimuli/quartz.png,attended_1,348.75,326.25,False
1,stimuli/travelmug.png,stimuli/bulldozer.png,1,stimuli/bulldozer.png,unattended,11.25,146.25,False
2,stimuli/razor04a.png,stimuli/mailbox02.png,2,stimuli/razor04a.png,unattended,168.75,123.75,False
3,stimuli/desktopcomputer.png,stimuli/greywolf.png,2,stimuli/greywolf.png,attended_2,213.75,236.25,False
4,stimuli/zebra.png,stimuli/penguin.png,1,stimuli/zebra.png,attended_3,123.75,326.25,False
5,stimuli/corkscrew03a.png,stimuli/radio03a.png,1,stimuli/corkscrew03a.png,attended_1,56.25,258.75,False
6,stimuli/maraca02a.png,stimuli/pintofbeer.png,2,stimuli/pintofbeer.png,attended_2,168.75,56.25,False
7,stimuli/chessknight03b.png,stimuli/fryingpan01.png,1,stimuli/chessknight03b.png,attended_3,123.75,303.75,False
8,stimuli/hen.png,stimuli/pylon01.png,1,stimuli/hen.png,attended_3,146.25,191.25,False
9,stimuli/pear01.png,stimuli/scrubbingbrush05a.png,2,stimuli/scrubbingbrush05a.png,attended_3,123.75,146.25,False


In [7429]:
# export my excel files for the working memory and ltm memory task
file_name_WM = 'conditions_079.xlsx'
participant_trials.to_excel(file_name_WM, index=False)

file_name_LTM = 'conditions_ltm_079.xlsx'
conditions_ltm_.to_excel(file_name_LTM, index=False)